## 어프라이어리 알고리즘

- 장점: 코드 작성하기 쉽다.
- 단점: 큰 데이터 집합일 수록 느려질 수 있다. ( DB를 Depth 만큼 호출해야 하기 때문.)
- 수치형/명목형 값에 활용 가능.
- 집합형태의 데이터 유형에 적합.
- 자주 나오는 아이템 집합과 아이템 간의 연관 규칙을 찾는데 사용.

* Support(지지도) : 모든 데이터 집합에서 아이템 집합(1개 이상)의 비율
* Confidence(신뢰도) : A 라는 아이템에서 B로 선택되는 확률,즉 신뢰도 = (A->B)의 지지도 / (A의 지지도)

In [29]:
import numpy as np

In [30]:
def loadDataSet():
    return [[1,3,4],[2,3,5],[1,2,3,5],[2,5]]

In [31]:
def createC1(dataSet):
    C1 = []                       # 유일한 값들을 저장하는데 사용.
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:  # C1에 해당 item 이 없으면 추가!
                C1.append([item])
    C1.sort()
    return map(frozenset, C1)   # frozenset : 내용물을 바꿀수 없게 하는 함수. 고정된 집합으로 만들어줌.
                                # 고정 집합을 사용하는 이유 : 해당 집합을 Dictionary의 키처럼 사용하기 위해 

[Frozenset 자세한 설명 링크 !클릭!](https://www.programiz.com/python-programming/methods/built-in/frozenset)

In [112]:
def scanD(D, Ck, minSupport):   # D: dataset / Ck: 후보 집합 리스트 / minSupport : 최소 지지도
    ssCnt = {}
    
    for tid in D:
        print(tid)
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    
    numItems = float(len(D))

    retList = []
    supportData = {}
    
    for key in ssCnt:
        support = ssCnt[key] / numItems     # 모든 원소에 대해 지지도를 측정.
        if support >= minSupport:           # 정한 최소 지지도 보다 크다면 저장.
            retList.insert(0, key)
        
        supportData[key] = support
        
    return retList, supportData


In [119]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [120]:
ab = createC1(dataSet)
C1 = list(ab) # C1 데이터 확인용.
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [121]:
cd = map(set, dataSet)
D = list(cd) # D 데이터 확인용
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [123]:
L1, suppData0 = scanD(D, C1, 0.5)
L1
print(suppData0)

{1, 3, 4}
{2, 3, 5}
{1, 2, 3, 5}
{2, 5}
{frozenset({1}): 0.5, frozenset({3}): 0.75, frozenset({4}): 0.25, frozenset({2}): 0.75, frozenset({5}): 0.75}


In [126]:
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)                  # Lk의 원소 갯수 파악
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            
            if L1 == L2:
                retList.append(Lk[i] | Lk[j]) # 합집합
                
    return retList

In [127]:
def apriori(dataSet, minSupport = 0.5):
    C1 = list(createC1(dataSet))
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
        
    return L, supportData

In [131]:
L, suppData = apriori(dataSet)
L

{1, 3, 4}
{2, 3, 5}
{1, 2, 3, 5}
{2, 5}
{1, 3, 4}
{2, 3, 5}
{1, 2, 3, 5}
{2, 5}
{1, 3, 4}
{2, 3, 5}
{1, 2, 3, 5}
{2, 5}
{1, 3, 4}
{2, 3, 5}
{1, 2, 3, 5}
{2, 5}


[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [133]:
L[0]

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [134]:
L[1]

[frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})]

In [136]:
aprioriGen(L[0], 2) # 6개의 결합이 나오면 apriori에서 scanD에 의해 2개가 걸러짐.

[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]

In [137]:
L, suppData = apriori(dataSet, minSupport = 0.7) # 모든 트랜잭션의 70%에서 발생하는 아이템이 무엇인지 알기 위해 설정.
L

{1, 3, 4}
{2, 3, 5}
{1, 2, 3, 5}
{2, 5}
{1, 3, 4}
{2, 3, 5}
{1, 2, 3, 5}
{2, 5}
{1, 3, 4}
{2, 3, 5}
{1, 2, 3, 5}
{2, 5}


[[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})], []]

In [138]:
def generateRules(L, supportData, minConf = 0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
                
    return bigRuleList

In [139]:
def calcConf(freqSet, H, supportData, br1, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet - conseq, " --> ", conseq, " conf:", conf)
            br1.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [140]:
def rulesFromConseq(freqSet, H, supportData, br1, minConf = 0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, br1, minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, br1, minConf)

In [141]:
L, suppData = apriori(dataSet, minSupport=0.5)

{1, 3, 4}
{2, 3, 5}
{1, 2, 3, 5}
{2, 5}
{1, 3, 4}
{2, 3, 5}
{1, 2, 3, 5}
{2, 5}
{1, 3, 4}
{2, 3, 5}
{1, 2, 3, 5}
{2, 5}
{1, 3, 4}
{2, 3, 5}
{1, 2, 3, 5}
{2, 5}


In [142]:
rules = generateRules(L, suppData, minConf=0.7)

frozenset({5})  -->  frozenset({2})  conf: 1.0
frozenset({2})  -->  frozenset({5})  conf: 1.0
frozenset({1})  -->  frozenset({3})  conf: 1.0


In [143]:
rules

[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

In [144]:
rules2 = generateRules(L, suppData, minConf=0.5)

frozenset({3})  -->  frozenset({2})  conf: 0.6666666666666666
frozenset({2})  -->  frozenset({3})  conf: 0.6666666666666666
frozenset({5})  -->  frozenset({3})  conf: 0.6666666666666666
frozenset({3})  -->  frozenset({5})  conf: 0.6666666666666666
frozenset({5})  -->  frozenset({2})  conf: 1.0
frozenset({2})  -->  frozenset({5})  conf: 1.0
frozenset({3})  -->  frozenset({1})  conf: 0.6666666666666666
frozenset({1})  -->  frozenset({3})  conf: 1.0
frozenset({5})  -->  frozenset({2, 3})  conf: 0.6666666666666666
frozenset({3})  -->  frozenset({2, 5})  conf: 0.6666666666666666
frozenset({2})  -->  frozenset({3, 5})  conf: 0.6666666666666666


In [146]:
rules2

[(frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]